<h1><center> K-Fold Analysis </center></h1>

LinkedIn Learning: https://www.linkedin.com/learning/nlp-with-python-for-machine-learning-essential-training/cross-validation-and-evaluation-metrics?u=78163626<br>
https://machinelearningmastery.com/k-fold-cross-validation/<br>
https://scikit-learn.org/stable/modules/cross_validation.html

## *Written by Nathanael Hitch*

<div style="color:purple">Background in classification and other metric in:</div>
<ul style="color:purple">
    <li>NLP_Logisitic-Regression.ipynb</li>
    <li>Random_Forest.ipynb</li>
</ul>

In previous looks at NLP sentiment models, there have been ways of analysing how good a model is when using testing data. For example:

- Accuracy: percentage of the total predictions that are completely correct.

# What is it?

It is when "the full data set is divided into *k*-subsets (*k* being a number) with the holdout method repeated *k* times. Each time, one of the subsets is used to test the model while all the other subsets are put together to train the model".

For a **5 fold validation**, k = 5, with a test set of 10,000 examples:

1. The test set is split into **5** subsets.
2. The first subset is put aside to test; the other 4 subsets train the model.
3. The accuracy (metric) for the model is determined by using the testing subset.
4. Each subset is used as a testing subset, with the other 4 training the model to be tested.
5. Each time the accuracy score is attained from each model.
6. The full array of avergae scores is output, along with the average of the model's accuracy.

You can see what difference between the lowest and highest score compared with the model's average score.<br>
Be aware that even small drops in accuracy can cause big changes in the results, e.g. a business setting where there are millions of pounds involved.

## Usefulness

Testing the model over a number of differenct sets will give a better indication about how the model will perform in the real-world.<br>
Depending on how big the differences between the averages are, we understand how well the model will work when put into production.

# Example

We can use sklearn to import in the K_Fold API for ease of use. Using the KFold object, you can see the split in the data set:

In [3]:
from numpy import array
from sklearn.model_selection import KFold

data = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6])
# data sample

kfold = KFold(3, True, 1)
# prepare cross validation

for train, test in kfold.split(data):
    print('train: %s, test: %s' % (data[train], data[test]))
# enumerate splits

train: [0.1 0.4 0.5 0.6], test: [0.2 0.3]
train: [0.2 0.3 0.4 0.6], test: [0.1 0.5]
train: [0.1 0.2 0.3 0.5], test: [0.4 0.6]
